In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from utils import refnis_to_arrondissment, get_arrondissement_from_code, prov_to_region
from utils import arrond_to_prov, get_ref_df
from place_name_mappings import col_names

from sklearn.impute import SimpleImputer 

pd.set_option('display.max_colwidth', 200)
pd.set_option("display.precision", 2)

In [9]:
def weighted_avg(series, weights):
    mask = ~series.isna()
    series_valid = series[mask]
    weights_valid = weights[mask]
    
    wsum = weights_valid.sum()
    if wsum == 0 or np.isnan(wsum):
        return np.nan
    return (series_valid * weights_valid).sum() / wsum

def make_weighted(col, n_col):
    return lambda g: weighted_avg(g[col], g[n_col])

def weighted_agg(group):
    return pd.Series({
        "n_1": group["n_1"].sum(),
        "n_2": group["n_2"].sum(),
        "n_3": group["n_3"].sum(),
        "q1_1": weighted_avg(group["q1_1"], group["n_1"]),
        "q2_1": weighted_avg(group["q2_1"], group["n_1"]),
        "q3_1": weighted_avg(group["q3_1"], group["n_1"]),
        "q1_2": weighted_avg(group["q1_2"], group["n_2"]),
        "q2_2": weighted_avg(group["q2_2"], group["n_2"]),
        "q3_2": weighted_avg(group["q3_2"], group["n_2"]),
        "q1_3": weighted_avg(group["q1_3"], group["n_3"]),
        "q2_3": weighted_avg(group["q2_3"], group["n_3"]),
        "q3_3": weighted_avg(group["q3_3"], group["n_3"]),
    })

cols_to_group = ["year","period", "n_1","q1_1","q2_1", "q3_1", "n_2", "q1_2", "q2_2", "q3_2", "n_3", "q1_3", "q2_3", "q3_3"]

#geo="refnis" # level_dict[level]
#
#df_agg = (
#    df.groupby(geo,group_keys=False)[cols_to_group]
#    .apply(weighted_agg)
#    .reset_index()
#    .sort_values(by="year")
#)

In [10]:
excell_file = "FR_immo_statbel_trimestre_par_commune.xlsx"
sheet       = "Par commune"
df          = get_ref_df(excell_file,sheet)

df["arrond"]         = df.refnis.apply(refnis_to_arrondissment)
df["arrondissement"] = df.arrond.apply(get_arrondissement_from_code)
df["province"]       = df.arrond.apply(arrond_to_prov)
df["region"]         = df.province.map(prov_to_region)

df.drop(["n_0","q1_0", "q2_0", "q3_0"],axis=1, inplace=True)


In [11]:
df.head()

,refnis,commune,year,period,n_1,q2_1,q1_1,q3_1,n_2,q2_2,q1_2,q3_2,n_3,q2_3,q1_3,q3_3,arrond,arrondissement,province,region
0,11001,AARTSELAAR,2010,Q1,25,247000,220000,265000,4,NaN,NaN,NaN,9,NaN,NaN,NaN,11,ARRONDISSEMENT D'ANVERS,10000,02000
1,11001,AARTSELAAR,2010,Q2,20,251250,209000,275000,5,NaN,NaN,NaN,11,NaN,NaN,NaN,11,ARRONDISSEMENT D'ANVERS,10000,02000
2,11001,AARTSELAAR,2010,Q3,18,250000,230000,275000,3,NaN,NaN,NaN,9,NaN,NaN,NaN,11,ARRONDISSEMENT D'ANVERS,10000,02000
3,11001,AARTSELAAR,2010,Q4,21,245000,205000,280000,7,NaN,NaN,NaN,13,NaN,NaN,NaN,11,ARRONDISSEMENT D'ANVERS,10000,02000
4,11001,AARTSELAAR,2011,Q1,18,280000,220000,360000,5,NaN,NaN,NaN,10,NaN,NaN,NaN,11,ARRONDISSEMENT D'ANVERS,10000,02000


In [12]:
def get_quartile(p0: float, q1: float, q3: float) -> int:
    if p0 < q1:
        return 1   
    elif p0 <= q3:
        return 2   
    else:
        return 3   

def closest_quartile(p0: float, q1: float, q2: float, q3: float) -> int:
    quartiles = [q1, q2, q3]
    distances = [abs(p0 - q) for q in quartiles]
    return distances.index(min(distances)) + 1  # +1 to get 1,2,3

def get_prices(area_df, t, cols ):
    return area_df.loc[ (df.year==t[0]) & (df.period ==t[1]),cols].values[0]

level_dict = {0: "refnis", 1: "arrond", 2: "province", 3: "region"}
levels = list(level_dict.values())

In [29]:
def price_predict_1(full_df, p0, t0: tuple[int,str], refnis: str , immo_type: int,  t1: tuple[int,str], level=0):

    immo_name = col_names[f"q1_{immo_type}"].split()[0]
    print(f"Price at time {t0} is {p0}")
    commune = df.loc[df["refnis"]==refnis, "commune"].iloc[0]

    cols = [f"n_{immo_type}",f"q1_{immo_type}", f"q2_{immo_type}", f"q3_{immo_type}"]
    df_commune = full_df.loc[(df.refnis==refnis) ].copy()
    
    n, q1, q2, q3 = get_prices(df_commune, t0, cols)
    if pd.isna(q1) or n==0:
        print(f"No data for {immo_name} in {commune} for time {t0}, going to check upper {levels[level+1]}.\n")
        return np.nan
    
    
    quart = closest_quartile(p0, q1, q2, q3)
    print(f"This is in quartile {quart}")
    col = f"q{quart}_{immo_type}"
    
    p_quart_before = get_prices(df_commune, t0, col)
    p_quart_after  = get_prices(df_commune, t1, col)
    print(f"{commune} Quart {quart} Price at time {t0} is {p_quart_before}")
    print(f"{commune} Quart {quart} Price at time {t1} is {p_quart_after}")
    
    
    if pd.isna(p_quart_after):
        print(f"No data for {immo_name} in {commune} for time {t1}, going to check upper {levels[level+1]}.\n")
        return np.nan
    p1 = p0 * p_quart_after/p_quart_before

    print(f"Price at time {t1} is {p1}")
    return p1


In [30]:
p0 = 1
t0 = (2011,"Q2")
refnis = 11001
t1 = (2011,"Q2")
immo_type = 1
level=0

p1 = price_predict_1(df, p0, t0, refnis, 1, t1)

p0 = 1
t0 = (2010,"Q1")
refnis = 11001
t1 = (2025,"Q1")
immo_type = 1


p1 = price_predict_1(df, p0, t0, refnis, immo_type, t1)

Price at time (2011, 'Q2') is 1
No data for Attached in AARTSELAAR for time (2011, 'Q2'), going to check upper arrond.

Price at time (2010, 'Q1') is 1
This is in quartile 1
AARTSELAAR Quart 1 Price at time (2010, 'Q1') is 220000
AARTSELAAR Quart 1 Price at time (2025, 'Q1') is 330000
Price at time (2025, 'Q1') is 1.5


In [15]:
p0 = 1
t0 = (2010,"Q1")
refnis = 11001
t1 = (2025,"Q1")
immo_type = 1

commune = df[df.refnis==refnis].commune.iloc[0]
arrondissement = df[df.refnis==refnis].arrondissement.iloc[0]
arond = refnis_to_arrondissment(refnis)

print(f"{commune} in {arrondissement}")

immo_name = col_names[f"q1_{immo_type}"].split()[0]
print(f"Price at time {t0} is {p0}")

cols = [f"n_{immo_type}",f"q1_{immo_type}", f"q2_{immo_type}", f"q3_{immo_type}"]
df_commune = df.loc[(df.refnis==refnis) ].copy()
df_commune_arrond = df.loc[(df.arrond==arond) ]

df_arond = (
    df_commune_arrond.groupby(["arrond","year","period"],group_keys=False)[cols_to_group]
    .apply(weighted_agg).reset_index().sort_values(by=["year","period"])
    )

n, q1, q2, q3 = get_prices(df_arond, t0, cols)
if pd.isna(q1) or n==0:
    print(f"No data for {immo_name} in {commune} for time {t0}, going to check upper {levels[level+1]}")
    #return np.nan

quart = closest_quartile(p0, q1, q2, q3)
print(f"This is in quartile {quart}")
col = f"q{quart}_{immo_type}"

p_quart_before = get_prices(df_arond, t0, col)
p_quart_after  = get_prices(df_arond, t1, col)
print(f"{arrondissement} Quart {quart} Price at time {t0} is {p_quart_before}")
print(f"{arrondissement} Quart {quart} Price at time {t1} is {p_quart_after}")
#
#
if pd.isna(p_quart_after):
    print(f"No data for {immo_name} in {commune} for time {t1}, going to check upper {levels[level+1]}")
    #return np.nan
p1 = p0 * p_quart_after/p_quart_before
#
print(f"Price at time {t1} is {p1}")

AARTSELAAR in ARRONDISSEMENT D'ANVERS
Price at time (2010, 'Q1') is 1
This is in quartile 1
ARRONDISSEMENT D'ANVERS Quart 1 Price at time (2010, 'Q1') is 158119.08931698775
ARRONDISSEMENT D'ANVERS Quart 1 Price at time (2025, 'Q1') is 297726.3337893297
Price at time (2025, 'Q1') is 1.882924668206668


In [31]:
def price_predict_2(df, p0, t0: tuple[int,str], refnis: str , immo_type: int,  t1: tuple[int,str], level=0):
    commune = df[df.refnis==refnis].commune.iloc[0]
    arrondissement = df[df.refnis==refnis].arrondissement.iloc[0]
    arond = refnis_to_arrondissment(refnis)
    
    print(f"{commune} in {arrondissement}")
    
    immo_name = col_names[f"q1_{immo_type}"].split()[0]
    print(f"Price at time {t0} is {p0}")
    
    cols = [f"n_{immo_type}",f"q1_{immo_type}", f"q2_{immo_type}", f"q3_{immo_type}"]
    df_commune_arrond = df.loc[(df.arrond==arond) ]
    
    df_arond = (
        df_commune_arrond.groupby(["arrond","year","period"],group_keys=False)[cols_to_group]
        .apply(weighted_agg).reset_index().sort_values(by=["year","period"])
        )
    
    n, q1, q2, q3 = get_prices(df_arond, t0, cols)
    if pd.isna(q1) or n==0:
        print(f"No data for {immo_name} in {commune} for time {t0}, going to check upper {levels[level+1]}")
        return np.nan
    
    quart = closest_quartile(p0, q1, q2, q3)
    print(f"This is in quartile {quart}")
    col = f"q{quart}_{immo_type}"
    
    p_quart_before = get_prices(df_arond, t0, col)
    p_quart_after  = get_prices(df_arond, t1, col)
    print(f"{arrondissement} Quart {quart} Price at time {t0} is {p_quart_before}")
    print(f"{arrondissement} Quart {quart} Price at time {t1} is {p_quart_after}")
    
    
    if pd.isna(p_quart_after):
        print(f"No data for {immo_name} in {commune} for time {t1}, going to check upper {levels[level+1]}")
        return np.nan
    p1 = p0 * p_quart_after/p_quart_before
    
    print(f"Price at time {t1} is {p1}")
    return p1

p0 = 1
t0 = (2010,"Q1")
refnis = 11001
t1 = (2025,"Q1")
immo_type = 1

price_predict_2(df,p0, t0, refnis, 1, t1)

AARTSELAAR in ARRONDISSEMENT D'ANVERS
Price at time (2010, 'Q1') is 1
This is in quartile 1
ARRONDISSEMENT D'ANVERS Quart 1 Price at time (2010, 'Q1') is 158119.08931698775
ARRONDISSEMENT D'ANVERS Quart 1 Price at time (2025, 'Q1') is 297726.3337893297
Price at time (2025, 'Q1') is 1.882924668206668


np.float64(1.882924668206668)

In [ ]:
def price_predict_3(df, p0, t0: tuple[int,str], refnis: str , immo_type: int,  t1: tuple[int,str], level=0):
    commune = df[df.refnis==refnis].commune.iloc[0]
    arrondissement = df[df.refnis==refnis].arrondissement.iloc[0]
    province  = df[df.refnis==refnis].province.iloc[0]
    arond = refnis_to_arrondissment(refnis)
    
    print(f"{commune} in {arrondissement}")
    
    immo_name = col_names[f"q1_{immo_type}"].split()[0]
    print(f"Price at time {t0} is {p0}")
    
    cols = [f"n_{immo_type}",f"q1_{immo_type}", f"q2_{immo_type}", f"q3_{immo_type}"]
    df_commune_arrond = df.loc[(df.arrond==arond) ]
    
    df_arond = (
        df_commune_arrond.groupby(["arrond","year","period"],group_keys=False)[cols_to_group]
        .apply(weighted_agg).reset_index().sort_values(by=["year","period"])
        )
    
    n, q1, q2, q3 = get_prices(df_arond, t0, cols)
    if pd.isna(q1) or n==0:
        print(f"No data for {immo_name} in {commune} for time {t0}, going to check upper {levels[level+1]}")
        return np.nan
    
    quart = closest_quartile(p0, q1, q2, q3)
    print(f"This is in quartile {quart}")
    col = f"q{quart}_{immo_type}"
    
    p_quart_before = get_prices(df_arond, t0, col)
    p_quart_after  = get_prices(df_arond, t1, col)
    print(f"{arrondissement} Quart {quart} Price at time {t0} is {p_quart_before}")
    print(f"{arrondissement} Quart {quart} Price at time {t1} is {p_quart_after}")
    
    
    if pd.isna(p_quart_after):
        print(f"No data for {immo_name} in {commune} for time {t1}, going to check upper {levels[level+1]}")
        return np.nan
    p1 = p0 * p_quart_after/p_quart_before
    
    print(f"Price at time {t1} is {p1}")
    return p1

In [17]:
display(df_arond.head(3))
display(df_commune.head(4))

,arrond,year,period,n_1,n_2,n_3,q1_1,q2_1,q3_1,q1_2,q2_2,q3_2,q1_3,q2_3,q3_3
0,11,2010,Q1,1243.0,323.0,1218.0,158119.09,198184.76,247359.02,288185.90,371702.99,516322.65,102048.79,130590.61,163135.74
1,11,2010,Q2,1321.0,376.0,1431.0,160858.56,203320.10,250437.55,284029.41,383980.97,514449.83,102458.07,130643.59,169526.11
2,11,2010,Q3,1376.0,407.0,1285.0,168405.59,209626.26,260977.08,266252.60,357301.30,485150.97,106964.90,134967.80,170161.88


,refnis,commune,year,period,n_1,q2_1,q1_1,q3_1,n_2,q2_2,q1_2,q3_2,n_3,q2_3,q1_3,q3_3,arrond,arrondissement,province,region
0,11001,AARTSELAAR,2010,Q1,25,247000,220000,265000,4,NaN,NaN,NaN,9,NaN,NaN,NaN,11,ARRONDISSEMENT D'ANVERS,10000,02000
1,11001,AARTSELAAR,2010,Q2,20,251250,209000,275000,5,NaN,NaN,NaN,11,NaN,NaN,NaN,11,ARRONDISSEMENT D'ANVERS,10000,02000
2,11001,AARTSELAAR,2010,Q3,18,250000,230000,275000,3,NaN,NaN,NaN,9,NaN,NaN,NaN,11,ARRONDISSEMENT D'ANVERS,10000,02000
3,11001,AARTSELAAR,2010,Q4,21,245000,205000,280000,7,NaN,NaN,NaN,13,NaN,NaN,NaN,11,ARRONDISSEMENT D'ANVERS,10000,02000


In [ ]:
def price_predict_recurse(full_df, p0, t0: tuple[int,str], geo_code: str , immo_type: int,  t1: tuple[int,str], level=0):

    print(f"Price at time {t0} is {p0}")
    #levels=levels[level:]
    #print(levels)
    geo= levels[level:][0]
    cols = [f"n_{immo_type}",f"q1_{immo_type}", f"q2_{immo_type}", f"q3_{immo_type}"]
    df_commune = full_df.loc[(df[geo]==geo_code) ].copy()

    df_agg = (
        area_df.groupby([geo,"year","period"],group_keys=False)[cols_to_group]
        .apply(weighted_agg)
        .reset_index().sort_values(by=["year","period"])
    )
    
    n, q1, q2, q3 = get_prices(df_agg, t0, cols)
    if pd.isna(q1) or n==0:
        print(f"No data for {col_names[f"q1_{immo_type}"].split()[0]} in {commune} for time {t0}, going to check upper {levels[level+1]}")
        return np.nan
    
    print(f"n_trans: {n}, q1: {q1}, q2: {q2}, q3: {q3}")
    
    quart = closest_quartile(p0, q1, q2, q3)
    print(f"This is in quartile {quart}")

    col = f"q{quart}_{immo_type}"
    
    p_quart_before = get_prices(df_commune, t0, col)
    p_quart_after  = get_prices(df_commune, t1, col)
    print(f"{commune} Quart {quart} Price at time {t0} is {p_quart_before}")
    print(f"{commune} Quart {quart} Price at time {t1} is {p_quart_after}")
    
    
    if pd.isna(p_quart_after):
        print(f"No data for {col_names[f"q1_{immo_type}"].split()[0]} in {commune} for time {t1}, going to check upper {levels[level+1]}")
        return np.nan
    p1 = p0 * p_quart_after/p_quart_before
    return p1

price_predict_1(df, 8000, (2010,"Q1"), refnis, 1, (2025,"Q1"))

Price at time (2010, 'Q1') is 8000
This is in quartile 1
AARTSELAAR Quart 1 Price at time (2010, 'Q1') is 220000
AARTSELAAR Quart 1 Price at time (2025, 'Q1') is 330000
Price at time (2025, 'Q1') is 12000.0


12000.0